In [10]:
import sys
import os
import numpy as np
from astropy.table import QTable, Table, vstack
import fnmatch
import matplotlib.pyplot as plt
from scipy.integrate import simps
import pickle 
import glob
import time,math
import clmm
import GCRCatalogs
from GCR import GCRQuery
import clmm.dataops
from clmm.dataops import compute_tangential_and_cross_components, make_radial_profile, make_bins
from clmm.galaxycluster import GalaxyCluster
import clmm.utils as u
from clmm import Cosmology
from clmm.support import mock_data as mock
cosmo = Cosmology(H0 = 71.0, Omega_dm0 = 0.265 - 0.0448, Omega_b0 = 0.0448, Omega_k0 = 0.0)
import iminuit
from iminuit import Minuit
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy.table import Table, QTable, hstack
from astropy import units as u
from astropy.coordinates import SkyCoord, match_coordinates_sky, Angle, match_coordinates_3d
cosmo_astropy = FlatLambdaCDM(H0=71.0, Om0=0.265, Ob0 = 0.0448)
import mysql
from mysql.connector import Error
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
def load(filename, **kwargs):
    """Loads GalaxyCluster object to filename using Pickle"""
    with open(filename, 'rb') as fin:
        return pickle.load(fin, **kwargs)

# Data

## Load dc2dr6 data

In [12]:
object_basic_cuts = [
    GCRQuery('extendedness > 0'),     # Extended objects
    GCRQuery('clean'), # The source has no flagged pixels (interpolated, saturated, edge, clipped...) 
                           # and was not skipped by the deblender
    GCRQuery('xy_flag == 0'),                                      # Flag for bad centroid measurement
    GCRQuery('ext_shapeHSM_HsmShapeRegauss_flag == 0'),            # Error code returned by shape measurement code
    GCRQuery((np.isfinite, 'ext_shapeHSM_HsmShapeRegauss_sigma')), # Shape measurement uncertainty should not be NaN
    ]

object_properties_cuts = [
    #GCRQuery('snr_i_cModel > 10'),                              # SNR > 10
    GCRQuery('mag_r_cModel < 28'),                                 # cModel imag brighter than 24.5
    GCRQuery('ext_shapeHSM_HsmShapeRegauss_resolution >= 0.4'), # Sufficiently resolved galaxies compared to PSF
    GCRQuery('ext_shapeHSM_HsmShapeRegauss_sigma <= 0.4'),      # Shape measurement errors reasonable
    # New cut on blendedness:
    GCRQuery('blendedness < 10**(-0.375)'),                      # Avoid spurious detections and those contaminated by blends
    GCRQuery('photoz_mean > 0.3'),
    GCRQuery('photoz_mean < 1')
    ]

In [13]:
quantity_label = ['id', 'ra', 'dec', 'mag_i_cModel','snr_i_cModel', 'mag_i', 'mag_r_cModel']
quantity_wanted_HSM = ['ext_shapeHSM_HsmShapeRegauss_e2',
                       'ext_shapeHSM_HsmShapeRegauss_e1',
                       'ext_shapeHSM_HsmShapeRegauss_sigma', 
                       'ext_shapeHSM_HsmShapeRegauss_resolution',
                      'mcal_g1', 'mcal_g2', 'mcal_gauss_g1','mcal_gauss_g1', 'mcal_psf_g1', 'mcal_psf_g2']
quantity_wanted_photozs = ['photoz_mean']